In [ ]:
# default_exp products

In [ ]:
#hide
# !pip install -q nbdev lambdasdk s3bz

In [ ]:
#hide
USER=None
PW=None

# Products

> interact with product apis

In [ ]:
#hide
from nbdev.showdoc import *
import logging

In [ ]:
#export
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda, InvocationType
from villaProductSdk.schema import Event, Response
from typing import Optional, List
import ujson as json
from nicHelper.wrappers import add_method
from nicHelper.exception import errorString
from awsSchema.apigateway import Event, Response
from dataclasses_json import dataclass_json
from dataclasses import dataclass
import bz2,  boto3, base64, logging

In [ ]:
#hide
logging.basicConfig(level=logging.INFO)

In [ ]:
#export
class FunctionNames:
  '''determine function and resources name based on branchName'''
  def __init__(self, branchName:str = 'dev-manual'):
    self.branchName = branchName
  dumpToS3 = lambda self: f'product-dump-s3-{self.branchName}'
  updateProduct = lambda self: f'product-update-{self.branchName}'
  updateS3 = lambda self: f'product-update-s3-{self.branchName}'
  singleQuery = lambda self: f'product-get-{self.branchName}'
  allQuery = lambda self: f'product-get-all-{self.branchName}'
  inputBucket = lambda self: f'input-product-bucket-{self.branchName}'
  inventoryBucket = lambda self: f'product-bucket-{self.branchName}'
  listQuery = lambda self: f'product-get-list-{self.branchName}'
  

In [ ]:
#export
class ProductSdk:
  '''
    the main class for interacting with product endpoint
    user/pw are optional
  '''
  def __init__(self, 
               branch:str = 'dev-manual', 
               user:Optional[str] = None, 
               pw:Optional[str] = None,
               region:str = 'ap-southeast-1'):
    self.branchName = branch
    self.functionNames = FunctionNames(branchName = branch)
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    
  @staticmethod
  def returnLambdaResponse(lambdaResponse:dict):
    try:
      return Response.fromDict(lambdaResponse).body
    except:
      logging.exception(f'error parsing body, perhaps there is no body in response\
      response is {lambdaResponse}')
      logging.error(lambdaResponse)
      
  @staticmethod
  def printFirst(inputDict:dict):
    return next(iter(inputDict.items()))
  
  def generalInvoke(self, functionName, payload):
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = payload
    )
    try:
      response:Response = Response.fromDict(lambdaResponse)
    except:
      print('unable to parse response')
      print(lambdaResponse)
      raise Exception(errorString())
    if response.statusCode == 200:
      return response.body
    else:
      print('error')
      return response.body
    return self.returnLambdaResponse(lambdaResponse)
    
  def updateWithS3(self, data, 
                   inputKeyName = 'input-data-name', 
                   invocationType = InvocationType.event,
                   user= None, pw= None):
    # put users if not specified
    user = user or self.user; pw = pw or self.pw
    
    # extract function name and inputbucket name
    inputBucketName = self.functionNames.inputBucket()
    functionName = self.functionNames.updateS3()
    logging.info(f'bucket is {inputBucketName}')
    
    # save data to s3
    S3.save(key = inputKeyName, 
            objectToSave = data , 
            bucket = inputBucketName,
            user=user, pw=pw)
    logging.info(f'data is saved to s3, invoking ingestion function')
    
    # call lambda function
    inputValue = Event(body = json.dumps({ 'key': inputKeyName })).to_dict()
    logging.info(f'input to lambda is {inputValue}')
    lambdaResponse = self.lambdaClient.invoke(
      functionName= functionName ,
      input=inputValue, 
      invocationType= invocationType )
    logging.info(f'lambdaResponse is {lambdaResponse}')
    if invocationType == 'Event': return "successfully sent event, please watch your slack"
    if lambdaResponse: return self.returnLambdaResponse(lambdaResponse)

  
  def allQuery(self):
    functionName = self.functionNames.allQuery()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    url = Response.fromDict(lambdaResponse).body['url']
    result = Requests.getContentFromUrl(url)
    return result
  def syncS3(self):
    '''force s3 to sync with the newly input data'''
    functionName = self.functionNames.dumpToS3()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    return self.returnLambdaResponse(lambdaResponse)


# Testing

In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
from pprint import pprint
import boto3
import pandas as pd


## generate dummy data for testing

In [ ]:
#Dummy Data
sampleProducts = [{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0235141', 'iprcode': '0235141', 'oprcode': '0235141', 'ordertype': 'Y', 'pr_abb': 'EEBOO-PZCT3-PUZZLE', 'pr_active': 'Y', 'pr_cgcode': '08', 'pr_code': '0235141', 'pr_dpcode': '19', 'pr_engname': 'EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3', 'pr_ggcode': '113', 'pr_market': 'eeboo,PUZZLE-PZCT3', 'pr_name': 'EEBOO-PZCT3-ตัวต่อนับเลข ANIMAL COUNTING_3ED', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM1979    ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0217153', 'iprcode': '0217153', 'oprcode': '0217153', 'ordertype': 'Y', 'pr_abb': 'COCOA LOCO MILK CHOC', 'pr_active': 'Y', 'pr_cgcode': '98', 'pr_code': '0217153', 'pr_dpcode': '28', 'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_ggcode': '003', 'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL', 'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_puqty': '24', 'pr_sa_method': '1', 'pr_sucode1': 'F1222     ', 'pr_suref3': 'S', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0182223', 'iprcode': '0182223', 'oprcode': '0182223', 'ordertype': 'Y', 'pr_abb': 'CIRIO PIZZASSIMO 400', 'pr_active': 'Y', 'pr_cgcode': '06', 'pr_code': '0182223', 'pr_dpcode': '06', 'pr_engname': 'CIRIO PIZZASSIMO 400G.', 'pr_ggcode': '004', 'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '2589      ', 'pr_suref3': 'C', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0124461', 'iprcode': '0124461', 'oprcode': '0124461', 'ordertype': 'Y', 'pr_abb': 'NEW CHOICE LYCHEE', 'pr_active': 'Y', 'pr_cgcode': '02', 'pr_code': '0124461', 'pr_dpcode': '02', 'pr_engname': 'NEW CHOICE LYCHEE', 'pr_ggcode': '003', 'pr_market': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_name': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '695       ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]


## Create main class object

In [ ]:
sdk = ProductSdk(branch = 'dev-manual')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Upload s3 data

In [ ]:
#hide

In [ ]:
%%time
result = sdk.updateWithS3(
    sampleProducts,
    invocationType = InvocationType.event
  )
result

INFO:root:bucket is input-product-bucket-dev-manual
INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:data is saved to s3, invoking ingestion function
INFO:root:input to lambda is {'body': '{"key":"input-data-name"}', 'headers': {}, 'statusCode': 200}
INFO:root:lambdaResponse is True


CPU times: user 51.1 ms, sys: 11.2 ms, total: 62.4 ms
Wall time: 212 ms


'successfully sent event, please watch your slack'

### test with lots of data

In [ ]:
data = list(pd.read_csv('./sampleData/products.csv').fillna('none').astype('string').T.to_dict().values())

/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,7,10,20,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
%%time
# data[0]
# try with 100 rows
result = sdk.updateWithS3(
    data[:1000],
    invocationType = InvocationType.event
  )
result

INFO:root:bucket is input-product-bucket-dev-manual
INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:data is saved to s3, invoking ingestion function
INFO:root:input to lambda is {'body': '{"key":"input-data-name"}', 'headers': {}, 'statusCode': 200}
INFO:root:lambdaResponse is True


CPU times: user 419 ms, sys: 3.95 ms, total: 423 ms
Wall time: 545 ms


'successfully sent event, please watch your slack'

## Query Single Product

In [ ]:
#export
@add_method(ProductSdk)
def querySingleProduct(self, iprcode = '0171670', user=None, pw=None):
  '''query a single product'''
  #extract function name
  functionName = self.functionNames.singleQuery()
  query = {'iprcode': iprcode}
  try:
    inputValue = Event.getInput(query)
    lambdaResponse = self.lambdaClient.invoke( functionName = functionName , input = inputValue )
    return self.returnLambdaResponse(lambdaResponse)
  except:
    print('calling lambda failed')


In [ ]:
result = sdk.querySingleProduct('0171670')
result

{'0171670': {'cprcode': '0171670',
  'iprcode': '0171670',
  'oprcode': '0171670',
  'ordertype': 'Y',
  'pr_abb': 'JIRAPAT YOUNG KALE 2',
  'pr_active': 'Y',
  'pr_cgcode': '05',
  'pr_code': '0171670',
  'pr_dpcode': '19',
  'pr_engname': 'JIRAPAT YOUNG KALE 200 G.',
  'pr_ggcode': '057',
  'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_puqty': '1.00',
  'pr_sa_method': '1',
  'pr_sucode1': 'CM845',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1',
  'pr_country_th': 'none',
  'pr_country_en': 'none',
  'pr_keyword_th': 'none',
  'pr_keyword_en': 'none',
  'pr_filter_th': 'none',
  'pr_filter_en': 'none',
  'online_category_l1_th': 'none',
  'online_category_l1_en': 'none',
  'online_category_l2_th': 'none',
  'online_category_l2_en': 'none',
  'online_category_l3_th': 'none',
  'online_category_l3_en': 'none',
  'content_en': '0171670 JIRAPAT YOUNG KALE 200 G.',
  'content_th': 'JIRAPAT YOUNG KALE 200 G.',
  'hema_brand_th':

In [ ]:
## failure query product notn found
result = sdk.querySingleProduct('3203290')
result

{'error': 'product not found'}

## All Query

In [ ]:
result = sdk.allQuery()
sdk.printFirst(result)

('0217153',
 {'0217153': {'cprcode': '0217153',
   'iprcode': '0217153',
   'oprcode': '0217153',
   'ordertype': 'Y',
   'pr_abb': 'COCOA LOCO MILK CHOC',
   'pr_active': 'Y',
   'pr_cgcode': '98',
   'pr_code': '0217153',
   'pr_dpcode': '28',
   'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_ggcode': '003',
   'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL',
   'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_puqty': '24.00',
   'pr_sa_method': '1',
   'pr_sucode1': 'F1239',
   'pr_suref3': 'S',
   'prtype': 'I',
   'psqty': '1',
   'pstype': '1',
   'pr_country_th': '',
   'pr_country_en': 'United Kingdom',
   'pr_keyword_th': '',
   'pr_keyword_en': '',
   'pr_filter_th': '',
   'pr_filter_en': '',
   'online_category_l1_th': '',
   'online_category_l1_en': '',
   'online_category_l2_th': '',
   'online_category_l2_en': '',
   'online_category_l3_th': '',
   'online_category_l3_en': '',
   'villa_category_l1_en': 'Dry Grocery',
   'villa_category_l2_en'

## Query list of items

In [ ]:
#export
@dataclass_json
@dataclass
class ProductsFromList:
  iprcodes: List[str]

In [ ]:
#export
@add_method(ProductSdk)
def queryList(self,iprcodes:List[str])->List[dict]:
  return self.generalInvoke(functionName=self.functionNames.listQuery() ,payload=Event.getInput({'iprcodes': iprcodes}))
  

In [ ]:
iprcodes = ['0217153','203915','0000009']
sdk.queryList(iprcodes)

[{'0217153': {'cprcode': '0217153',
   'iprcode': '0217153',
   'oprcode': '0217153',
   'ordertype': 'Y',
   'pr_abb': 'COCOA LOCO MILK CHOC',
   'pr_active': 'Y',
   'pr_cgcode': '98',
   'pr_code': '0217153',
   'pr_dpcode': '28',
   'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_ggcode': '003',
   'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL',
   'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_puqty': '24.00',
   'pr_sa_method': '1',
   'pr_sucode1': 'F1239',
   'pr_suref3': 'S',
   'prtype': 'I',
   'psqty': '1',
   'pstype': '1',
   'pr_country_th': '',
   'pr_country_en': 'United Kingdom',
   'pr_keyword_th': '',
   'pr_keyword_en': '',
   'pr_filter_th': '',
   'pr_filter_en': '',
   'online_category_l1_th': '',
   'online_category_l1_en': '',
   'online_category_l2_th': '',
   'online_category_l2_en': '',
   'online_category_l3_th': '',
   'online_category_l3_en': '',
   'villa_category_l1_en': 'Dry Grocery',
   'villa_category_l2_en': 'Grocery',

## Trigger s3 sync

In [ ]:
%%time
response = sdk.syncS3()
response

In [ ]:
#hide
from nbdev.export import *
notebook2script()

In [ ]:
%time
import requests
from io import BytesIO
from PIL import Image

##large Image
url = 'http://d19oj5aeuefgv.cloudfront.net'
sku = '0189194'
Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])

In [ ]:
%time
url = 'https://d1vl5j0v241n75.cloudfront.net'
sku = '0189194'
Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])